### ¿Qué es model-complexity-reducer (mcr) y para que sirve?

La herramienta mcr esta diseñada para maximizar el tradeoff entre la complejidad de algoritmos GBT y su performance. 
En este caso, por complejidad nos vamos a referir al número de operaciones que necesita el algoritmo para realizar una inferencia (número de árboles y profundidad de estos).

Esta herramienta esta diseñada específicamente para que el modelo resultante sea manejable en el paradigma de ZKML pero su funcionalidad puede ser extendida a cualquier problema donde:

- Es necesario que el peso del modelo sea mínimo, por ejemplo para aplicaciones móviles.
- Son necesarios tiempos de inferencia mínimos para aplicaciones low latency.
- Queremos comprobar si hemos creado un modelo muy complejo y uno más sencillo nos daría el mismo performance (o incluso mejor). 
- El número de pasos necesarios para realizar la inferencia debe ser inferior a X (como la restricción que nos impone a día de hoy el paradigma de ZKML).

Una vez entendida las posibles aplicaciones de esta herramienta vamos a ver como funciona.

### ¿Cómo funciona mcr?

La mejor forma de entender como funciona nuestro algoritmo de reducción de la dimensionalidad es explicar paso a paso las operaciones que realiza:

- Dado un modelo, recupera el tipo de modelo que es y los parámetros importantes de este.
- En función de estos parámetros, ajusta el espacio de búsqueda, de forma que las arquitecturas que tratará de encontrar serán más ligeras que la orginal.
- Realiza un proceso de optimización bayesiana, donde añade como posibles hiperparámetros:
    - Parámetros propios del modelo ajustados al nuevo espacio de búsqueda.
    - Transformaciones de los datos de entrada usando diferentes técnicas de reduccionalidad.
- Tras obtener los mejores parámetros para una métrica de evaluación dada, devuelve:
    - Un transformer con la transformación que se le deberá aplicar al conjunto de test para generar la inferencia.
    - El nuevo modelo ajustado.


### MCR en el contexto de Giza

Dentro del mundo de ZKML sabemos que el coste de ejecutar una ZK Proof es mucho más elevado que el coste de generar una inferencia normal, debido al proceso criptográfico que está involucrado.
En este contexto, se ha desarrollado mcr con el objetivo de crear modelos que puedan ser transpilables a otros modelos. Estos nuevos modelos podrán ser interpretados por otros programas (en nuestro caso Cairo) y generar ZK Proofs.

El stack de Giza, como veremos en el ejemplo end_to_end_using_giza_stack,ipynb, es capaz de coger un modelo serializado con nuestra herramienta serializer y hacer todo el proceso de transpilación + generación de la inferencia verificable por nosotros. Lo unico que tenemos que asegurarnos es de no crear un modelo demasiado grande!

In [1]:
# For this example, it is necessary to have both xgboost and lightgbm installed, but it is not necessary to have all packages installed to use giza_mlutils. 
# For this reason, we include this cell to ensure the notebook works correctly.

!pip install xgboost
!pip install lightgbm


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


En este sencillo tutorial, solo vamos a ver como se usa el paquete, sin entrar en detalles de performance. Para ver estos detalles, mira el notebook end_to_end_using_giza_stack.ipynb

In [1]:
import lightgbm as lgbm
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split

data = load_diabetes()
X, y = data.data, data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

n_estimators = 1200
max_depth = 8

lgbm_reg = lgbm.LGBMRegressor(n_estimators=n_estimators, max_depth=max_depth)
lgbm_reg.fit(X, y)

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000210 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 682
[LightGBM] [Info] Number of data points in the train set: 442, number of used features: 10
[LightGBM] [Info] Start training from score 152.133484
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

LGBMRegressor(max_depth=8, n_estimators=1200)

In [2]:
lgbm_reg.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': 8,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 1200,
 'n_jobs': None,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [3]:
from giza_mlutils.model_reducer import mcr

In [4]:
model, transformer = mcr(model = lgbm_reg,
                         X_train = X_train,
                         y_train = y_train, 
                         X_eval = X_test, 
                         y_eval = y_test, 
                         eval_metric = 'rmse', 
                         can_we_transform_your_features = True)

Adjusting max_depth to new range: [3, 8]
Adjusting num_leaves to new range: [20, 31]
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000286 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 595
[LightGBM] [Info] Number of data points in the train set: 353, number of used features: 10
[LightGBM] [Info] Start training from score 153.736544
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [4]:
model, transformer = mcr(model = lgbm_reg,
                         X_train = X_train,
                         y_train = y_train, 
                         X_eval = X_test, 
                         y_eval = y_test, 
                         eval_metric = 'rmse', 
                         can_we_transform_your_features = False)

Adjusting max_depth to new range: [3, 8]
Adjusting num_leaves to new range: [20, 31]
[LightGBM] [Warning] early_stopping_round is set=10, early_stopping_rounds=10 will be ignored. Current value: early_stopping_round=10
[LightGBM] [Warning] feature_fraction is set=0.4389006955136503, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4389006955136503
[LightGBM] [Warning] bagging_fraction is set=0.349425478714389, subsample=1.0 will be ignored. Current value: bagging_fraction=0.349425478714389
[LightGBM] [Warning] min_data_in_leaf is set=89, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=89
[LightGBM] [Warning] feature_fraction is set=0.4389006955136503, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4389006955136503
[LightGBM] [Warning] bagging_fraction is set=0.349425478714389, subsample=1.0 will be ignored. Current value: bagging_fraction=0.349425478714389
[LightGBM] [Warning] min_data_in_leaf is set=89, min_child_sa

KeyError: 'dimensionality_reduction'

Ya tenemos nuestro modelo y nuestro transformer para hacer inferencias!